To fazendo alguns testes separados aqui nesse notebook, vou tentar fazer esse teste de exemplo que tem no pdf novo que subi, que tá no moodle. Acho que as duas principais coisas que temos que resolver é conseguir executar uma arvore de decisão simples com esses dados de benchmark e fazer um cross validation com algum outro tipo de dado, pq o meu do exercício anterior não deu certo.

Vou seguir fazendo esse exemplo aqui, depois se puder sobe teu trabalho de crossvalidation pra gente já começar a ajustar caso não esteja fazendo aquela divisão estratificada que falamos aquele dia. 

In [19]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from __future__ import print_function

dataset = pd.read_csv("C:/Users/renan/Desktop/UFRGS/Machine Learning/Trabalho1/random-forests-1/dadosBenchmark_validacaoAlgoritmoAD.csv") 

dir(dataset)

print ('Banco com %d amostras e %d colunas\n' % dataset.shape)

print ('Tabela sumarizando as colunas do banco\n')
dataset.describe()

Banco com 14 amostras e 5 colunas

Tabela sumarizando as colunas do banco



,Tempo,Temperatura,Umidade,Ventoso,Joga
count,14,14,14,14,14
unique,3,3,2,2,2
top,Ensolarado,Amena,Normal,Falso,Sim
freq,5,6,7,8,9


In [30]:
n_samples, n_features = dataset.shape

In [24]:
unique_vals(dataset, 0)

{&#39;J&#39;, &#39;T&#39;, &#39;U&#39;, &#39;V&#39;}

In [22]:
def class_counts(rows):
    
    counts = {}  
    for row in rows:
        
        Joga = row[-1]
        if Joga not in counts:
            counts[Joga] = 0
        counts[Joga] += 1
    return counts

class_counts(dataset)

{&#39;o&#39;: 2, &#39;a&#39;: 2, &#39;e&#39;: 1}

In [26]:
def gini(rows):
  
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [27]:
def entropy_criterion(data, labels):
  """ Entropy
  Parameters
  ----------
  data: numpy array-like = [n_samples, n_features]
  labels: numpy array-like, shape = [n_samples]
  
  Return
  ------
  entropy: float
  """
  classes = np.unique(labels)
  
  s = 0
  for c in classes:
    p = np.mean(labels == c)
    s -= p * np.log(p)
    
  return s